## 1.Data Profilling

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.axes as ax
import sklearn
#print(sklearn.__version__)

In [ ]:
### Read all the domcuments artribute types as required ###
### Change File Location as needed ###
CS_DS = pd.read_csv("C:/Users/USER/5. SCG/preML_v5b_010622.csv",
        dtype={"tax_id": str, "cust_name": str,"mind_sect": str, "input1_tax_id": str
                            #"max_overdue_days": float,
              })

In [ ]:
CS_DS.head()

In [ ]:
### Drop Columns that won't be used in machine learning algorithm ###
CS_DS2 = CS_DS.drop(columns=['tax_id','cust_name','mind_sect','input1_tax_id','class_max'])
CS_DS2

In [6]:
### Data Profile for Input Data ###
pd.set_option('display.max_columns', None)
pd.set_option('float_format', '{:f}'.format)
describer = CS_DS2.describe(percentiles = [.5]).T
list1 = np.count_nonzero(CS_DS2, axis=0).tolist()
describer['non_zeros'] = list1
describer['Name'] = describer.index

describer = describer[['Name','count','mean','std','min','50%','max','non_zeros']]
describer

,Name,count,mean,std,min,50%,max,non_zeros
max_overdue_days,max_overdue_days,1244.000000,49.299839,107.200728,-42.000000,18.000000,2242.000000,1132
average_overdue_days,average_overdue_days,1244.000000,17.753079,75.354339,0.000000,5.600000,2232.200000,1111
overdue_payments,overdue_payments,1244.000000,96.551447,525.799406,0.000000,11.000000,13366.000000,1111
total_payments,total_payments,1244.000000,910.383441,2114.060288,1.000000,150.500000,30352.000000,1244
before_due_bucket,before_due_bucket,1244.000000,32.491961,189.192383,0.000000,1.000000,3466.000000,779
...,...,...,...,...,...,...,...,...
OD_outstanding_std,OD_outstanding_std,1244.000000,1612062.842366,6341953.682936,0.000000,0.000000,64454129.840000,214
OD_utilize_max,OD_utilize_max,1244.000000,0.126141,0.300820,0.000000,0.000000,1.621627,238
OD_utilize_mean,OD_utilize_mean,1244.000000,0.101097,0.256194,0.000000,0.000000,1.008004,238
OD_utilize_std,OD_utilize_std,1244.000000,0.018720,0.058092,0.000000,0.000000,0.475033,214


## 2.Modeling

In [7]:
### Cross-Validation [Stratified K-Fold] ###
from statistics import mean, stdev
from sklearn.model_selection import train_test_split

### Logistic Regression module ###
from sklearn.linear_model import LogisticRegression

### Separate Data into inputs vs desired output
#X = CS_DS2.iloc[:, 0:122].values
#y = CS_DS2.iloc[:, 122].values
X = CS_DS2.drop(['class_max_binary'], axis = 'columns').values #Train set
y = CS_DS2.class_max_binary.values#Test set
#X
#y

In [8]:
### Default Logistic Regression Settings
log_reg = LogisticRegression(max_iter=100)
### Default Random Forest Settings

### USE PARAMETERS FROM RANDOMSEARCH HERE
#LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
#          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
#          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
#          verbose=0, warm_start=False)

### Split input and output into training and hidden(testing) data
X_train, X_hidden, y_train, y_hidden = train_test_split(CS_DS2.drop(['class_max_binary'], axis = 'columns').values,
                                                        CS_DS2.class_max_binary.values,
                                                        stratify=y, test_size=0.15,random_state=0)

#### StratifiedKFold - Cross validation
- For making a good combination of both yes and no in the train set
- Use stratisfied whenever we have a huge imbalance dataset


In [9]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, balanced_accuracy_score

### Manual stratified k-fold cross-validation
skf = StratifiedKFold(n_splits=10)

lst_B_accu_stratified = []
lst_accu_stratified = []

for train_index, test_index in skf.split(X_train, y_train):
    x_train_fold, x_test_fold = X_train[train_index], X_train[test_index]
    y_train_fold, y_test_fold = y_train[train_index], y_train[test_index]
    
    log_reg.fit(x_train_fold, y_train_fold) 
    #Without Kfold, will be  log_reg.fit(X_train, y_train)
    y_pred_fold = log_reg.predict(x_test_fold)
    
    lst_accu_stratified.append(accuracy_score(y_test_fold, y_pred_fold))
    lst_B_accu_stratified.append(balanced_accuracy_score(y_test_fold,y_pred_fold))
    
    #print(lst_B_accu_stratified)

C:\Users\USER\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\USER\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i 

##### Stratified K-fold evaluation

In [10]:
### Cross-Validation Evaluation [Evaluated against validation sets not hidden set]     
print('\nMaximum Balanced Accuracy That can be obtained from this model is:',
      max(lst_B_accu_stratified)*100, '%')
print('\nMinimum Balanced Accuracy:',
      min(lst_B_accu_stratified)*100, '%')
print('\nOverall Balanced Accuracy:',
      mean(lst_B_accu_stratified)*100, '%')
print('==================================')
print('Length Check')
print('full_data',len(X),len(y))
print('training',len(X_train),len(y_train))
print('Hidden',len(X_hidden),len(y_hidden))
print('folds','train',len(x_train_fold),len(y_train_fold),'test',len(x_test_fold),len(y_test_fold))


Maximum Balanced Accuracy That can be obtained from this model is: 73.17857142857143 %

Minimum Balanced Accuracy: 61.81818181818181 %

Overall Balanced Accuracy: 68.38354341736694 %
Length Check
full_data 1244 1244
training 1057 1057
Hidden 187 187
folds train 952 952 test 105 105


## 3.Model Evaluations
- Confusion matrix
- accuracy score
- ROC
- Precision_Recall

In [11]:
### Model Evaluation [Evaluated against Hidden Test Set / test set {Unseen Data}] ###
from sklearn.metrics import (classification_report, confusion_matrix, accuracy_score, balanced_accuracy_score)
from sklearn.metrics import (roc_auc_score, plot_roc_curve, roc_curve, precision_recall_curve,plot_precision_recall_curve,average_precision_score, 
                             PrecisionRecallDisplay)

y_pred_train = log_reg.predict(X_train)
y_pred_hidden = log_reg.predict(X_hidden)

print("==== TRAINING ====")
print(confusion_matrix(y_train,y_pred_train))
print(classification_report(y_train,y_pred_train))
print("balanced accuracy:",balanced_accuracy_score(y_train,y_pred_train))

print("\n==== HIDDEN ====\n")
print(confusion_matrix(y_hidden,y_pred_hidden))
print(classification_report(y_hidden,y_pred_hidden))
print("Balanced Accuracy:",balanced_accuracy_score(y_hidden,y_pred_hidden))

print("ROC AUC Score:",roc_auc_score(y_hidden,log_reg.predict_proba(X_hidden)[:, 1]))

print("Average Precision Score:",average_precision_score(y_hidden,log_reg.predict_proba(X_hidden)[:, 1]))

==== TRAINING ====
[[316 187]
 [126 428]]
              precision    recall  f1-score   support

           0       0.71      0.63      0.67       503
           1       0.70      0.77      0.73       554

    accuracy                           0.70      1057
   macro avg       0.71      0.70      0.70      1057
weighted avg       0.70      0.70      0.70      1057

balanced accuracy: 0.7003968965987468

==== HIDDEN ====

[[50 39]
 [27 71]]
              precision    recall  f1-score   support

           0       0.65      0.56      0.60        89
           1       0.65      0.72      0.68        98

    accuracy                           0.65       187
   macro avg       0.65      0.64      0.64       187
weighted avg       0.65      0.65      0.64       187

Balanced Accuracy: 0.6431437743636781
ROC AUC Score: 0.7244897959183673
Average Precision Score: 0.7184658340396541


## 4. Feature Importance

In [ ]:
feature_names = CS_DS2.columns.tolist()

feature_importance = pd.DataFrame(feature_names, columns = ["feature"])
feature_importance["importance"] = pow(math.e, w)
feature_importance = feature_importance.sort_values(by = ["importance"], ascending=False)
 
from sklearn.linear_model import LogisticRegression
ax = feature_importance.plot.barh(x='feature', y='importance')
plt.show()

In [ ]:
### Feature Importance Asessment [from Random Forest]
feature_importance= log_reg.feature_importances_
#feature_importance= grid_RF_test.feature_importances_
sorted_idx = np.argsort(feature_importance)
pos = np.arange(sorted_idx.shape[0]) + 0.5
fig = plt.figure(figsize=(12, 20))
feature_names = CS_DS2.columns.tolist()

### Change plot settings as required ###
plt.subplot(1, 2, 1)
plt.barh(pos, feature_importance[sorted_idx], align="center")
plt.yticks(pos, np.array(feature_names)[sorted_idx])
plt.tick_params(axis='y', colors='white')
plt.title("Feature Importance (MDI)", color = 'white')
#sorted_idx
#feature_names

## 5.Hyper parameter selection

In [16]:
#### HYPER PARAMETER SELECTION [Randomized CV Search] ####
def param(score_type):
    from sklearn.model_selection import RandomizedSearchCV
    from sklearn.metrics import roc_auc_score
    
    solvers = ['newton-cg', 'lbfgs', 'liblinear']
    penalty = ['l2']
    c_values = [100, 10, 1.0, 0.1, 0.01]
    
    # Create the random grid
    random_grid = {'solver': solvers,
                   'penalty': penalty,
                   'C': c_values}
    print(random_grid)

    # Use the random grid to search for best hyperparameters
    # First create the base model to tune
    #RF_Model
    # Random search of parameters, using 3 fold cross validation or change to desried k value, 
    # search across 100 different combinations, and use all available cores
    log_reg_random = RandomizedSearchCV(estimator = log_reg, scoring =score_type, 
                param_distributions = random_grid, n_iter = 100, cv = 3, 
                verbose=2, random_state=42, n_jobs = -1)
    # Fit the random search model
    log_reg_random.fit(X_train, y_train)

    print(score_type,"\n",log_reg_random.best_params_)
    best_random = log_reg_random.best_estimator_
    print("ROC AUC Score:",roc_auc_score(y_hidden,log_reg_random.predict_proba(X_hidden)[:, 1]))
    print("ROC AUC Score:",roc_auc_score(y_hidden,best_random.predict_proba(X_hidden)[:, 1]))

#param('balanced_accuracy')
param('roc_auc')
#param('average_precision')

### SEE IF ROC_AUC SCORE HAS IMPROVED OR NOT ###

{'solver': ['newton-cg', 'lbfgs', 'liblinear'], 'penalty': ['l2'], 'C': [100, 10, 1.0, 0.1, 0.01]}
Fitting 3 folds for each of 15 candidates, totalling 45 fits


C:\Users\USER\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:278: UserWarning: The total space of parameters 15 is smaller than n_iter=100. Running 15 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   10.7s
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:   13.1s finished


roc_auc 
 {'solver': 'liblinear', 'penalty': 'l2', 'C': 100}
ROC AUC Score: 0.7058014216922724
ROC AUC Score: 0.7058014216922724


C:\Users\USER\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


In [15]:
### MODEL EVALUATION OF Settings from Randomized Search ###

### USE PARAMETERS FROM RANDOMSEARCH HERE
log_reg_model2 = LogisticRegression(solver = 'liblinear',
                                    penalty = 'l2',
                                    C = 100)

from sklearn.metrics import (classification_report, confusion_matrix, accuracy_score, 
                             balanced_accuracy_score)
from sklearn.metrics import (roc_auc_score, plot_roc_curve, roc_curve, precision_recall_curve, 
                             plot_precision_recall_curve,average_precision_score, 
                             PrecisionRecallDisplay)

log_reg_model2.fit(X_train, y_train)
y_pred_train = log_reg_model2.predict(X_train)
y_pred_hidden = log_reg_model2.predict(X_hidden)

print("==== TRAINING ====")
print(confusion_matrix(y_train,y_pred_train))
print(classification_report(y_train,y_pred_train))
print("balanced accuracy:",balanced_accuracy_score(y_train,y_pred_train))
print("\n==== HIDDEN ====\n")
print(confusion_matrix(y_hidden,y_pred_hidden))
print(classification_report(y_hidden,y_pred_hidden))
print("Balanced Accuracy:",balanced_accuracy_score(y_hidden,y_pred_hidden))
print("ROC AUC Score:",roc_auc_score(y_hidden,log_reg_model2.predict_proba(X_hidden)[:, 1]))
print("Average Precision Score:",average_precision_score(y_hidden,log_reg_model2.predict_proba(X_hidden)[:, 1]))


==== TRAINING ====
[[343 160]
 [118 436]]
              precision    recall  f1-score   support

           0       0.74      0.68      0.71       503
           1       0.73      0.79      0.76       554

    accuracy                           0.74      1057
   macro avg       0.74      0.73      0.73      1057
weighted avg       0.74      0.74      0.74      1057

balanced accuracy: 0.7344560794080284

==== HIDDEN ====

[[50 39]
 [29 69]]
              precision    recall  f1-score   support

           0       0.63      0.56      0.60        89
           1       0.64      0.70      0.67        98

    accuracy                           0.64       187
   macro avg       0.64      0.63      0.63       187
weighted avg       0.64      0.64      0.63       187

Balanced Accuracy: 0.632939692731025
ROC AUC Score: 0.7058014216922724
Average Precision Score: 0.6942823573762793


C:\Users\USER\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
